In [32]:
#Import Dependencies
import pandas as pd

In [33]:
state_data = 'publicAssistanceData/output_data/state_data/'
states = ["Alabama","Alaska","Arizona","Arkansas","California","Colorado",
  "Connecticut","Delaware","Florida","Georgia","Hawaii","Idaho","Illinois",
  "Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland",
  "Massachusetts","Michigan","Minnesota","Mississippi","Missouri","Montana",
  "Nebraska","Nevada","NewHampshire","NewJersey","NewMexico","NewYork",
  "NorthCarolina","NorthDakota","Ohio","Oklahoma","Oregon","Pennsylvania",
  "RhodeIsland","SouthCarolina","SouthDakota","Tennessee","Texas","Utah",
  "Vermont","Virginia","Washington","WestVirginia","Wisconsin","Wyoming", "DistrictofColumbia"]
dfs = []
for state in states:
    df = pd.read_csv(f'publicAssistanceData/output_data/state_data/{state}Public.csv')
    df['state'] = state
    dfs.append(df)
len(dfs)

51

In [34]:
#Extra cleaning of FEMA data for merging
def data_clean(df):
    #Remove statewide data
    
    #Changing date to year
    df = df.rename(columns={'disasterNumber':'disasterID', 'amountOblig':'FEMA$'})
    #Remove data before '03
    df = df[df['year'] >= 2003]
    df = df[df['year'] <= 2018]
    return df

In [35]:
clean_dfs = []
for df in dfs:
    data_clean(df)
    clean_dfs.append(df)
len(clean_dfs)

51

In [36]:
#concat all states
total_df = clean_dfs[0]
for df in clean_dfs:
   total_df = pd.concat([total_df, df])

In [37]:
#group data by incident type then county
total_group = total_df.groupby(['incidentType', 'state', 'county'])
total_df2 = pd.DataFrame({'incidentCount':total_group['disasterNumber'].count(), 'FEMA$':total_group.amountOblig.mean()})
total_df2 = total_df2.drop('Statewide', level=2)
total_df2.tail()

incidentCount         FEMA$
incidentType state  county                              
Tsunami      Hawaii Maui                 7  2.939823e+04
             Oregon Coos                 3  4.454609e+04
                    Curry                5  1.018568e+06
                    Lincoln              3  6.313122e+04
Volcano      Hawaii Hawaii              14  9.136473e+05

In [38]:
#Export new df into csv
total_df2.to_csv('publicAssistanceData/byDisasterandCounty.csv')